In [265]:
%%bash
# Install required packages
pip install pandas
pip install sentence-transformers

#Had to add these to make it work locally
pip install matplotlib
pip install seaborn
pip install sentence_transformers
pip install nltk

In [266]:
import os
import json
import re
from typing import List, Tuple, Dict, Union
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import importlib
from collections import Counter
import string

# import QSFNN
# importlib.reload(QSFNN)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cuda device


In [267]:
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, "../../data")
print(data_dir)

/home/ameli/cse517p-project/src/data_parsing/../../data


In [268]:
raw_dataset = pd.read_csv(
    f"{data_dir}/raw_data/nasa_transcripts.csv"
)

In [269]:
raw_dataset.head()

,mission,document,speaker_code,dialogue
0,apollo_17,AS17_TEC.pdf,CDR,Roger. The clock has started. We have yaw.
1,apollo_17,AS17_TEC.pdf,CDR,"Roger; tower. Yaw's complete. We're into roll,..."
2,apollo_17,AS17_TEC.pdf,CC,"Roger, Geno. Looking great. Thrust good on all..."
3,apollo_17,AS17_TEC.pdf,CDR,"Okay, babe. It 's looking good here."
4,apollo_17,AS17_TEC.pdf,CDR,Roll is complete. We are pitching.


In [270]:
dialogue = raw_dataset["dialogue"]

In [271]:
dialogue = dialogue.str.lower()
print(dialogue.head())

0           roger. the clock has started. we have yaw.
1    roger; tower. yaw's complete. we're into roll,...
2    roger, geno. looking great. thrust good on all...
3                 okay, babe. it 's looking good here.
4                   roll is complete. we are pitching.
Name: dialogue, dtype: object


In [272]:
# Combine all text in the DataFrame into a single string
all_text = ''.join(dialogue.astype(str).values.flatten())

# Count each character
char_counts = Counter(all_text)
print(char_counts)
print(len(char_counts))

to_remove = []

for char in char_counts:
    if char_counts[char] < 150:
        to_remove.append(char)


Counter({' ': 2049011, 'e': 887752, 't': 803702, 'o': 726946, 'a': 631592, 'i': 495660, 'n': 478271, 'r': 429283, 'h': 414336, 's': 407060, 'l': 321261, '.': 307345, 'u': 272136, 'd': 264813, 'y': 227812, 'g': 220862, 'c': 201774, 'w': 174847, ',': 168867, 'm': 168357, 'p': 143206, 'k': 138371, 'f': 123640, 'b': 115091, "'": 107150, '-': 89039, 'v': 80173, '0': 75514, '1': 51513, '2': 44407, '_': 37795, '5': 34234, '3': 32895, '4': 28397, '?': 27432, 'j': 24275, '6': 19258, ';': 18015, '7': 17305, '8': 14368, '9': 12991, ':': 9856, 'x': 9465, '*': 8922, 'q': 7425, 'z': 6531, '(': 6245, ')': 6140, '"': 5376, '/': 4889, '!': 2861, '°': 2129, ']': 1932, '[': 1602, '%': 891, '{': 878, '}': 600, '&': 563, '·': 512, '#': 327, '$': 284, '=': 208, '¢': 146, '£': 115, '¥': 111, '<': 106, '©': 85, '±': 82, '>': 82, '+': 63, '\\': 52, '~': 32, '×': 18, '^': 15, '@': 11, '®': 7, '½': 6, '¸': 6, '|': 6, '§': 3, '¼': 1, 'ù': 1})
82


In [273]:
dialogue.head()

0           roger. the clock has started. we have yaw.
1    roger; tower. yaw's complete. we're into roll,...
2    roger, geno. looking great. thrust good on all...
3                 okay, babe. it 's looking good here.
4                   roll is complete. we are pitching.
Name: dialogue, dtype: object

In [274]:

def clean_data(input):
    clean_dialogue = []
    joined = ''.join(to_remove)
    joined+="{}[]°"
    joined+=string.punctuation

    trans = str.maketrans('', '', joined)

    for line in input:
        cleaned = line.translate(trans)
        
        clean_dialogue.append(cleaned)

        
   
    return pd.DataFrame(clean_dialogue, columns=['dialogue'])


In [275]:
dialogue_clean = clean_data(dialogue)

dialogue_clean.head()

dialogue_clean.replace('', np.nan, inplace=True)
dialogue_clean.dropna(inplace=True) 

print(dialogue_clean)
train_end = int(0.7 * len(dialogue_clean))
dev_end = int(0.85 * len(dialogue_clean))



# Split the DataFrame
train_df = dialogue_clean.iloc[:train_end]
dev_df   = dialogue_clean.iloc[train_end:dev_end]
test_df  = dialogue_clean.iloc[dev_end:]

# Optional: check sizes
print(len(train_df), len(dev_df), len(test_df))



                                                 dialogue
0                 roger the clock has started we have yaw
1            roger tower yaws complete were into roll bob
2       roger geno looking great thrust good on all fi...
3                        okay babe it s looking good here
4                        roll is complete we are pitching
...                                                   ...
143722  330 169 15 08 03 15 31 hornet do you have an e...
143723  the condition of crew  4000  3500 feet on the ...
143724  okay our splashdown error isby latitude longit...
143725  apollo 11 at 1500 feet 08 c3 17 27 hornet horn...
143726   300 feet 08 03 17 56 swim 1 roger youre looki...

[142053 rows x 1 columns]
99437 21308 21308


In [276]:
out_dir = data_dir

train_df.to_csv(f"{data_dir}/parsed_data/train_nasa.csv", index=False)
dev_df.to_csv(f"{data_dir}/parsed_data/dev_nasa.csv", index=False)
test_df.to_csv(f"{data_dir}/parsed_data/test_nasa.csv", index=False)

In [277]:
star_trek_dialogue = pd.read_json(f"{data_dir}/raw_data/star_trek_raw_lines.json")
star_trek_dialogue = star_trek_dialogue["message"].str.lower()

star_trek_dialogue = clean_data(star_trek_dialogue)

star_trek_dialogue.replace('', np.nan, inplace=True)
star_trek_dialogue.dropna(inplace=True) 

star_trek_dialogue = pd.DataFrame(star_trek_dialogue)

In [278]:
star_trek_dialogue.head()

,dialogue
0,check the circuit
1,all operating sir
2,it cant be the screen then definitely somethin...
3,it could be these meteorites
4,no its something else theres still something o...


In [279]:
train_end = int(0.7 * len(star_trek_dialogue))
dev_end = int(0.85 * len(star_trek_dialogue))

# Split the DataFrame
train_df = star_trek_dialogue.iloc[:train_end]
dev_df   = star_trek_dialogue.iloc[train_end:dev_end]
test_df  = star_trek_dialogue.iloc[dev_end:]

# Optional: check sizes
print(len(train_df), len(dev_df), len(test_df))

189079 40517 40517


In [280]:
train_df.to_csv(f"{data_dir}/parsed_data/train_trek.csv", index=False)
dev_df.to_csv(f"{data_dir}/parsed_data/dev_trek.csv", index=False)
test_df.to_csv(f"{data_dir}/parsed_data/test_trek.csv", index=False)